# Integration of Cutting

## How to start grinder

0. mount shaftgrinder and cutter (Fräser) s.t. 6cm of cutter are visible
1. checkout branch 17-cutting-merge
2. catkin build your ws
3. connect arduino to your laptop and to 24 V (if Arduino is broken, see instructions below) 
4. connect shaftgrinder to 230 V and 24 V
5. ssh to robot PC
~~~
 roslaunch hrr_cobot_robot hrr_cobot_hw.launch use_rviz:=false tool_name:="shaftgrinder"
~~~
6. start arduino node
~~~
roslaunch hrr_ee_tools tool_controller.launch
~~~
(you should hear the valve)
7. walk through notebook

In [15]:
# packages needed for this tutorial
from pathlib import Path
# math and robotics
import numpy as np
import quaternion
from tqdm.notebook import tqdm, trange
import spatialmath as sm

# plotting 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt

# simulated robot content and helpers
import hrr_common as hrr_cm
from hrr_controllers import GimaticPinController
import hrr_cobot_robot as hrr_rob
from hrr_cobot_robot.tool_control import ArduinoToolControlInterface
from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common

# ROS
import rospy

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib notebook

In [16]:
hrr_common.set_ros_environment("127.0.0.1")


In [2]:
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://127.0.0.1:11311
# %env ROS_MASTER_URI=http://hrrcobotLinux54:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

env: ROS_MASTER_URI=http://127.0.0.1:11311
	!!!please check that the IPs below is identical!!!
	host hrrN3511rt2004 has IP 192.168.1.35, which should be identical to
vs:	env: ROS_IP=192.168.1.35


In [17]:
rospy.init_node("hrr_cobot_shaftgrinder")

In [3]:
rospy.set_param("/hrr_cobot/cmd_arduino", True) # to use arduino interface. If you want to use the shield, don't set
# this parameter. Defaults to new shield.

In [18]:
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

## TOOL CHANGER

In [5]:
GP = GimaticPinController.from_ros(cobot_prefix="/hrr_cobot")


In [6]:
GP.activate()


In [10]:
GP.set_to_start_state()  # open tool-changer


In [9]:
rospy.sleep(1.0)
GP.invert_start_state()  # close tool-changer
GP.deactivate()

In [ ]:
cobot.tool_changer.

In [4]:
cobot.move_to_joint_pose(cobot.q_calib)

## CUTTING TESTING

In [15]:
print(cobot.FK(cobot.q))

In [7]:
cobot.FK(shaftgrind_calib)

NameError: name 'shaftgrind_calib' is not defined

In [6]:
shaftgrind_calib = np.array([ 0.   ,   0.  ,   -1.5708,  1.5708,      1.5708 , 0.    ])

In [7]:
cobot.move_to_joint_pose(shaftgrind_calib)

[INFO]desired joint-pose is further than 0.5 (1.57) radiants from current, use stochastic motion planner
> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 94)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 95)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 96)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 98)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 98)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 98)
In HrRecyclerPerecptioPipeline (line 50)

calculating signed distance field ...
calculating signed distance field done
Elapsed time is 7.635991 seconds.
Elapsed time is 0.567391 seconds.


[WARN][/hrr_cobot_shaftgrinder] Planner is done and execution is running ...


[INFO]re-/deactivate controllers


In [4]:
cobot.tool_controller

[WARN]trying to access tool-controller without a current tool


In [19]:
cobot.change_tool("shaftgrinder")

True

In [19]:
cobot.change_tool("nothing")

True

In [21]:
cobot.close_tool_changer()

In [22]:
cobot.open_tool_changer(force=True)

In [ ]:
rospy.get_param("/hrr_cobot/tool_name") # to use arduino interface. If you want to use the shield, don't set
# this parameter. Defaults to new shield.

In [9]:
cobot.tool_controller.rpm = 30000

[WARN]maximum value is 25000. Set value accordingly.


In [21]:
cobot.tool_controller.rpm = 0

In [26]:
cobot.tool_controller.run_shaftgrinder(0.05, "slow")

In [7]:
cobot.tool_controller.enable_grinder() #sometimes needed to turn grinder on again

AttributeError: 'HrrCobotToolControlInterface' object has no attribute 'enable_grinder'

In [20]:
cobot.tool_controller.disable() 
#for safety you can disable the tool but then you have to set it to shaftgrinder again before you can start it

AttributeError: 'HrrCobotToolControlInterface' object has no attribute 'disable'

## How to test transformation/kinematic chain

1. set up as above but on/off does not necessarily have to work
2. run function definitions below for a single pose - if wanted, all 4 poses can be calculated in section 1.2.2
3. 


**TODO: Update toEE vector (depending on kinematic chain, test with absolute values)**

### Single Pose

In [6]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50,rpm=0.005):
    """
    Sends robot to given position.
    """
    
    cobot.move_to_pose(pose) #v_max,rot_max,rot_precision_threshold=0.001, pos_precision_threshold=0.001)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        if rpm != 0:
            cobot.tool_controller.rpm = rpm
            cobot.tool_controller.enable_grinder()

        cobot.update()

In [19]:
## sets a safe transformation for robot with shaftgrinder attached.

# Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0., 0.455498],
#                             [0, 0, 1, 0.0482934],
#                             [0.707107, 0.707107, 0, 0.4],
#                             [0, 0, 0, 1]])

xstart = 0.4166
ystart = -0.05628
zstart = 0.025

#normal (-0.1905, 0.1043, 0.9761)

Shaftgrinder_tf = np.array([[-1 ,       0 , 0,  0.4359 ],   
                           [0 ,0 ,-1   ,     -0.08     ], 
                           [0, -1 ,        0,  0.785 ]  ,  
                           [0,         0  ,       0   ,      1] ])

## sets translational vector
toEE = np.array([0, -0.156, 0.094])

In [34]:
print(cobot.T_E_C_robot.inv())
#print(Shaftgrinder_tf)

In [7]:
# use data from vision/task planner or set coordinates manually

xstart = 0.5 #determine start value next to cable (in start cutting pose)
ystart = 0.5
zstart =  0.2 #0.025

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstart,ystart,zstart], "pointA")

In [18]:
# calculates the pose from transformation, translation vector and given points

CuttingPoseA = hrr_cm.calc_EE_pre_pose(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                        safety_distance=0)
print(CuttingPoseA)

NameError: name 'toEE' is not defined

In [9]:
cobot.update_ee_tcp_tf()

In [8]:
cobot.set_tool_frame(3)

In [10]:
    def calc_EE_cutting_pose(B_normal, B_p_location, T_C_E=None, B_y_axis=np.r_[0., 1., 0.], depth=-5e-2):
        r"""Calculate cutting pose via cutting with the robot into the surface

        The surface is defined by its normal, while the ``safety_distance``
        describes the hovering height that the robot should be steered to

        Args:
            B_normal (np.ndarray): normal vector in base frame :math:`{}^{B}{\bf n} \in \mathbb{R}^{3}`
            B_p_location(np.ndarray): goal-position in base-frame :math:`{}^{B}{\bf p}_{BC} \in \mathbb{R}^{3}`.
            B_y_axis (np.ndarray, optional): reference y-axis to generate pose / right-hand CS in base-frame. Defaults to :math:`{}^{B}{\bf e}_{y}`.
            T_C_E(sm.SE3 or None, optional): homogeneous transformation from end-effector to tool-frame. Defaults to None.
            depth (float, optional): cutting depth. Defaults to -0.05.

        Returns:
            sm.SE3: pre-pose of end-effector wrt to base-frame.

        Raises:
            AssertionError: if safety_distance is negative
        """
        assert depth <= 0.0, "cutting depth is positive"
        return sm.SE3(B_normal * depth) @ hrr_cm.calc_EE_goal_pose(B_normal=B_normal, B_p_location=B_p_location,
                                                                   B_y_axis=B_y_axis, T_C_E=T_C_E)

In [11]:
CuttingPoseA = calc_EE_cutting_pose(B_normal=hrr_cm.vec32np(normalA.vector),
                                                     B_p_location=hrr_cm.vec32np(
                                                         pointA.vector),
                                                     T_C_E=cobot.T_E_C_robot.inv(),
                                                     B_y_axis=np.r_[0., 1., 0.],
                                                     depth=0)

In [12]:
print(CuttingPoseA)

In [22]:
print(cobot.q)

In [13]:
cobot.stochastic_move_to_pose(CuttingPoseA)

> In HrRecyclerPerecptioPipeline (line 18)
> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 94)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 95)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 96)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 98)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 98)
In HrRecyclerPerecptioPipeline (line 50)

> In gpmp2.test3Ddataset>add_obstacle (line 189)
In gpmp2.test3Ddataset (line 98)
In HrRecyclerPerecptioPipeline (line 50)

calculating signed distance field ...
calculating signed distance field done
Elapsed time is 6.724701 seconds.
Elapsed time is 0.339511 seconds.


[WARN][/hrr_cobot_shaftgrinder] Planner is done and execution is running ...


KeyboardInterrupt: 

In [15]:
print(cobot.q)

[-0.15175  0.51511 -1.96199 -0.       0.66449 -1.73504]


In [31]:
cobot.stop()

In [28]:
  shit = np.array([[-1       ,  0     ,    0     ,    0.5651],    
   [0    ,     1  ,       0  ,       0.1366 ],   
   [0,         0   ,     -1   ,      0.32  ],    
   [0,         0    ,     0    ,     1]])


In [31]:
cobot.goTo(cobot.q_calib, v_max=0.03)

In [39]:
MoveLoop(pose=cobot.FK(cobot.q_calib),v_max=0.02,rot_max=0.1,timeout=5000,rpm=0)

  0%|          | 0/500000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
MoveLoop(pose=CuttingPoseA,v_max=0.02,rot_max=0.1,timeout=5000,rpm=0)

  0%|          | 0/500000 [00:00<?, ?it/s]

### 4 Poses (as final setup)

In [15]:
def MoveLoop(pose,v_max=0.005,rot_max=0.005,timeout=50,rpm=0.005):
    """
    Sends robot to given position.
    """
    cobot.move_to_pose(pose,v_max,rot_max,rot_precision_threshold=0.001, pos_precision_threshold=0.001)
    T = int(timeout * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerr(f"robot in ERROR state")
        if rpm != 0:
            cobot.tool_controller.rpm = rpm
            cobot.tool_controller.enable_grinder()

        cobot.update()

In [16]:
## sets a safe transformation for robot with shaftgrinder attached.

Shaftgrinder_tf = np.array([[-0.707107, 0.707107, 0., 0.455498],
                            [0, 0, 1, 0.0482934],
                            [0.707107, 0.707107, 0, 0.653348],
                            [0, 0, 0, 1]])

## sets translational vector
toEE = np.array([0, -0.156, 0.094])

In [17]:
# use data from vision/task planner or set coordinates manually

xstart = 0.592829 #determine start value next to cable (in start cutting pose)
ystart = -0.229713
zstart =  0.367

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstart,ystart,zstart], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,-1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstart,ystart+0.031,zstart], "pointB")

In [19]:
PrePoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                        safety_distance=-0.05)

PrePoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=-0.05)

CuttingPoseA = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalA.vector),
                                        p_location=hrr_cm.vec32np(pointA.vector),
                                        C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                        safety_distance=-0)

CuttingPoseB = hrr_cm.calc_EE_pre_pose_neg(normal=hrr_cm.vec32np(normalB.vector),
                                       p_location=hrr_cm.vec32np(pointB.vector),
                                       C_p_CE=toEE, R_C_E=Shaftgrinder_tf[:3, :3],
                                       safety_distance=-0)

In [24]:
MoveLoop(pose=PrePoseA,v_max=0.2,rot_max=0.4,timeout=5000,rpm=0)
MoveLoop(pose=CuttingPoseA,v_max=0.06,rot_max=0.02,timeout=5000,rpm=5000)
MoveLoop(pose=CuttingPoseB,v_max=0.06,rot_max=0.02,timeout=5000,rpm=5000)
MoveLoop(pose=PrePoseB,v_max=0.2,rot_max=0.4,timeout=5000,rpm=5000)
cobot.tool_controller.rpm = 0
cobot.tool_controller.disable()

  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1645031720.934701]: reached goal pose at step 1358 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1645031772.888284]: reached goal pose at step 5191 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1645031818.294769]: reached goal pose at step 4537 / 500000


  0%|          | 0/500000 [00:00<?, ?it/s]

[INFO] [1645031833.883530]: reached goal pose at step 1553 / 500000


[WARN] [1645031833.890883]: minimum value is 3500 => set value to 0
[WARN] [1645031833.895191]: minimum value is 3500 => set value to 0


## How to call action service

1. set up as above
2. additionally start the server in PyCharm (for easy debugging) or just run the file 
~~~
python /hrr_cobot/hrr_cobot_robot/scripts/devel/cutting_skill.py
~~~
3. As device type and device center is needed, you need to start a dummy publisher for both (or adapt to action rostopics): 
~~~
python hrr_cobot/hrr_cobot_robot/scripts/devel/dummyDevCenterPub.py
python /hrr_cobot/hrr_cobot_robot/scripts/devel/dummyDevTypePub.py
~~~
4. As soon as rospy.spin() is running, you can set up the client and send a goal (see below)

* define the messages that should come from task planner for calling the action service
* points (coordinates) must come from vision
* normals should be similar to the ones here to avoid collisions. you can also leave them like this and don't consider vision data

In [14]:
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.5,0.5,0.2], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.5,0.55,0.2], "pointB")
## replace by vision data

In [ ]:
# use data from vision/task planner or set coordinates manually

xstart = 0.5 #determine start value next to cable (in start cutting pose)
ystart = 0.5
zstart =  0.2 #0.025

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[xstart,ystart,zstart], "pointA")

create the Action client and look for server. Returns "found" if the Server is up and running.

In [22]:
shaftgrinder_client=actionlib.SimpleActionClient('/cutting_server/cutting', CuttingAction)
shaftgrinder_client.wait_for_server()
print('found')

found


In [24]:
shaftgrinder_client=actionlib.SimpleActionClient('/hrr_cobot/grinding_action_srv_name', CuttingAction)
shaftgrinder_client.wait_for_server()
print('found')

KeyboardInterrupt: 

define goal - should be done/called by task planner/IBEC

In [23]:
goal=CuttingGoal()
goal.surface_normal_start=normalA
goal.surface_normal_end=normalB
goal.start_location=pointA
goal.end_location=pointB
goal.material.material_type=material
goal.timeout=5000
goal.thickness = 0.005 #not important. can be anything

print(goal)

surface_normal_start: 
  header: 
    seq: 0
    stamp: 
      secs: 1645009136
      nsecs: 337403297
    frame_id: "normalA"
  vector: 
    x: 0
    y: 0
    z: -1
surface_normal_end: 
  header: 
    seq: 0
    stamp: 
      secs: 1645009136
      nsecs: 340436697
    frame_id: "normalB"
  vector: 
    x: 0
    y: 0
    z: -1
start_location: 
  header: 
    seq: 0
    stamp: 
      secs: 1645009136
      nsecs: 337791919
    frame_id: "pointA"
  vector: 
    x: 0.4555
    y: 0.0159017
    z: 0.65335
end_location: 
  header: 
    seq: 0
    stamp: 
      secs: 1645009136
      nsecs: 341432571
    frame_id: "pointB"
  vector: 
    x: 0.4555
    y: -0.0540378
    z: 0.65335
material: 
  material_type: 1
thickness: 0.005
timeout: 5000


start actual execution by sending the goal

In [24]:
shaftgrinder_client.send_goal(goal)
shaftgrinder_client.wait_for_result()
shaftgrinder_client.get_result()

KeyboardInterrupt: 

if needed: you can cancel all goals

In [ ]:
shaftgrinder_client.cancel_all_goals()

## Arduino Fix:

Take new arduion and flash it with the following programm in Arduino IDE: 

File: 

~~~
/hrr_cobot/hrr_ee_tools/assets/general_ee_tools/general_ee_tools.ino
~~~

or copy content from cell below

In [ ]:
/*
  @file general_ee_tools.ino
  @author Annalena Daniels (a.daniels@tum.de)
  @author Klara Tomaskovic (klara.tomaskovic@gmail.com)
  @author Volker gabler (v.gabler@tum.de)
  @brief HR-Recycler Tool control script
  @version 0.1.1
  @date 2021-12-30
  This short arduino code enables an Arduino to control

*  * PIN  6: a Shaft grinder via PWM motor control using a 10V control line
*  * PIN 13: a vacuum gripper controlled via a pneumatic vacuum switch
*  * PIN 0-5 & 7-12: a screwdriver KOLVER pluto 10
  @copyright Copyright (c) 2021 TUM-LSR HR-Recycler Group
*/

//#define DEBUG TRUE // outcomment to compile with debugging options
#define USE_PROG58 // enable program 5 to 8
#include <ros.h>
#include <std_msgs/Bool.h>
#include <std_msgs/Int8.h>
#include <std_msgs/Float32.h>
#include <std_srvs/SetBool.h>
#include <std_srvs/Empty.h>

using std_srvs::SetBool;
using std_srvs::Empty;


#define SC_STOP 2  // STOP SCREWDRIVER PIN I/O
#define SC_START 3 // SCREWDRIVER START PIN I/O
#define SC_REVERSE 4 // SCREWDRIVER REVERSE PIN I/O
#define SC_PRG1 12 // SCREWDRIVER PROGRAM 1 PIN I/O
#define SC_PRG2 11 // SCREWDRIVER PROGRAM 2 PIN I/O
#define SC_PRG3 10 // SCREWDRIVER PROGRAM 3 PIN I/O
#define SC_PRG4 9 // SCREWDRIVER PROGRAM 4 PIN I/O
#define SC_PRG5 8 // SCREWDRIVER PROGRAM 5 PIN I/O
#define SC_PRG6 7 // SCREWDRIVER PROGRAM 6 PIN I/O
#define SC_PRG7 5 // SCREWDRIVER PROGRAM 7 PIN I/O
#define SC_PRG8 4 // SCREWDRIVER PROGRAM 8 PIN I/O 


#define GRINDER_SWITCH 6 // GRINDER PIN (needs PWM)
#define VACUUM_SWITCH 13 // VACUUM PIN


#define GR 3
#define SC 4
#define VC 5
#define OFF -2
#define NONE 0

// globals
bool grinder_on = false;
bool emergency = false;
bool vacuum_on = false;
bool screwdriver_on = false;
int16_t t_last_msg, t;
int16_t t_last_screwdriver_msg, t_last_vacuum_msg, t_last_grinder_msg;
int16_t T_timeout = 1000;
std_msgs::Int8 driver_state;
// Grinder Control
uint8_t PWM = 0;
float RPM = 0;
// screwdriver control
int8_t sc_prog = -1;
bool sc_start = false;
bool sc_reverse = false;
bool sc_stop = false;

// debug content
#ifdef DEBUG
std_msgs::Float32 debug_data0;
std_msgs::Float32 debug_data1;
#endif


/**
   @brief grinder control -> convert RPM to PWM

   @param rpm set value in rotations/minute
   @param min_pwm_i8 minimum pwm number. Values below are ignored, i.e. 0.
   @param max_pwm_i8 maximum pwm number. Values above are cut to 255.
   @return uint8_t pwm value to set grinder to speed of rpm
*/
uint8_t rpm_to_pwm(const float& rpm,
                   const uint8_t& min_pwm_i8 = 20,
                   const uint8_t& max_pwm_i8 = 255) {
  float max_rpm = 25000.0;
  float min_rpm = 3500.0;
  if (rpm < min_rpm) {
    return 0;
  } else if (rpm > max_rpm) {
    return max_pwm_i8;
  }
  float pwm_range = (float) (max_pwm_i8 - min_pwm_i8);
  float rpm_range = max_rpm - min_rpm;
  float rpm_to_pwm_coefficient = (max_rpm*(float)min_pwm_i8 - min_rpm * (float) max_pwm_i8 )/ rpm_range;
  float rpm_to_pwm_slope = pwm_range / rpm_range;
  uint8_t pwm = (uint8_t) ( rpm_to_pwm_slope * rpm + rpm_to_pwm_coefficient);
  if (pwm > max_pwm_i8) {
    return max_pwm_i8;
  } else if (pwm < min_pwm_i8) {
    return 0;
  } else {
    return pwm;
  }
}

/**
   @brief Set the grinder speed object

   @param grinder_pwm_speed ROS-msg as rmp speed value
*/
void set_grinder_speed(const std_msgs::Float32& grinder_rpm_speed) {
  RPM = grinder_rpm_speed.data;
  PWM = rpm_to_pwm(RPM); // float -> unsigned int8
  t_last_msg = t;
  t_last_grinder_msg = t;
}


/**
   @brief Set the screwdriver button cb object
   values:
    - 1 -> enable program with set motion direction
    - 2 -> enable program with reversed motion
    - default -> trigger stop
   @param button ROS-message as button
*/
void set_screwdriver_button_cb(const std_msgs::Int8& button) {
  if (screwdriver_on) {
    switch (button.data) {
      case 1:
        sc_start = true;
        sc_reverse = false;
        sc_stop = false;
        break;
      case -1:
        sc_start = false;
        sc_reverse = true;
        sc_stop = false;
        break;
      case 0:
        sc_start = false;
        sc_reverse = false;
        sc_stop = true;
      default:
        sc_start = false;
        sc_reverse = false;
        sc_stop = false;
        break;
    }
  }
  t_last_msg = t;
  t_last_screwdriver_msg = t;
}

/**
   @brief Set the screwdriver prog cb object

   ignore values outside of (1,8)
   @param button ROS-message for desired program
*/
void set_screwdriver_prog_cb(const std_msgs::Int8& button) {
  if (screwdriver_on) {
    if ((button.data > 0) && (button.data <= 8)) {
      sc_prog = button.data;
    }
  }
  t_last_msg = t;
  t_last_screwdriver_msg = t;
}



/**
   @brief shared function -> reset set values
*/
void reset_values() {
  emergency = false;
  t = 0;
  t_last_msg = -1;
  t_last_screwdriver_msg = -T_timeout;
  t_last_vacuum_msg = -T_timeout;
  t_last_grinder_msg = -T_timeout;
}

/**
   @brief shared function -> reset emergency state via ROS-service
*/
void reset_srv_cb(const Empty::Request & req, Empty::Response & res) {
  if (emergency) {
    reset_values();
  }
}


/**
   @brief helper function
   disable screwdriver mode if last screwdriver message is older than 0.5 seconds
*/
bool disable_sc() {
  if (t - t_last_screwdriver_msg > T_timeout) {
    screwdriver_on = false;
    return true;
  }
  return false;
}

/** @brief helper function
    disable vacuum gripper mode if last vacuum gripper message is older than 0.5 seconds
*/
bool disable_vc() {
  if (t - t_last_vacuum_msg > T_timeout) {
    vacuum_on = false;
    return true;
  }
  return false;
}

/** @brief helper function
    disable shaft grinder mode if last vacuum gripper message is older than 0.5 seconds
*/
bool disable_gr() {
  if (t - t_last_grinder_msg > T_timeout) {
    grinder_on = false;
    return true;
  }
  return false;
}


/** @brief shared function -> set control mode
    Set control mode via an Int8 ROS-msg
*/
void set_tool_mode_cb(const std_msgs::Int8& mode) {
  if (emergency) {
    return;
  }
  if (mode.data == NONE){
    grinder_on = false;
    screwdriver_on = false;
    vacuum_on = false;
    reset_values();
  } 
  // set to grinder if possible
  else if (mode.data == GR){
    if (screwdriver_on) {
      if (disable_sc()) {grinder_on = true;}
    } else if (vacuum_on) {
      if (disable_vc()) {grinder_on = true;}
    } else {grinder_on = true;}
    t_last_grinder_msg = t;
  } 
  // set to screwdriver mode if possible
  else if (mode.data == SC){
    if (grinder_on) {
      if (disable_gr()) {screwdriver_on = true;}
    } else if (vacuum_on) {
      if (disable_vc()) {screwdriver_on = true;}
    } else {screwdriver_on = true;}
    t_last_screwdriver_msg = t;
  } 
  // set to vacuum gripper mode if possible
  else if (mode.data == VC){
    if (grinder_on) {
        if (disable_gr()) {vacuum_on = true;}
      } else if (screwdriver_on) {
        if (disable_sc()) {vacuum_on = true;}
      } else {vacuum_on = true;}
      t_last_vacuum_msg = t;
  } else {
      emergency = true;
      grinder_on = false;
      screwdriver_on = false;
      vacuum_on = false;
  }
  t_last_msg = t;
}


ros::NodeHandle  nh;
// shared ROS-API
ros::ServiceServer<Empty::Request, Empty::Response> reset_srv("~reset", &reset_srv_cb);
ros::Subscriber<std_msgs::Int8> sub_tool_mode("~set_control_mode", &set_tool_mode_cb);
ros::Publisher pub_driver_status("~status", &driver_state);

// grinder control
ros::Subscriber<std_msgs::Float32> grinder_speed_sub("~grinder/speed", &set_grinder_speed);

// screwdriver control
ros::Subscriber<std_msgs::Int8> sub_screwdriver_button_cb("~screwdriver/motor", &set_screwdriver_button_cb);
ros::Subscriber<std_msgs::Int8> sub_screwdriver_mode_cb("~screwdriver/set_program", &set_screwdriver_prog_cb);

// debug out
#ifdef DEBUG
ros::Publisher pub_debug_data0("~debug/data0", &debug_data0);
ros::Publisher pub_debug_data1("~debug/data1", &debug_data1);
#endif
/**
   @brief Set the pins of the Arduino
*/
void set_pins()
{
  // set grinder
  pinMode(GRINDER_SWITCH, OUTPUT);
  analogWrite(GRINDER_SWITCH, 0);

  // set vacuum
  pinMode(VACUUM_SWITCH, OUTPUT);
  digitalWrite(VACUUM_SWITCH, LOW);

  // set screwdriver
  pinMode(SC_START, OUTPUT);
  pinMode(SC_REVERSE, OUTPUT);
  pinMode(SC_PRG1, OUTPUT);
  pinMode(SC_STOP, OUTPUT);
  pinMode(SC_PRG1, OUTPUT);
  pinMode(SC_PRG2, OUTPUT);
  pinMode(SC_PRG3, OUTPUT);
  pinMode(SC_PRG4, OUTPUT);

  digitalWrite(SC_START, LOW);
  digitalWrite(SC_REVERSE, LOW);  
  digitalWrite(SC_STOP, LOW);
  digitalWrite(SC_PRG1, LOW);
  digitalWrite(SC_PRG2, LOW);
  digitalWrite(SC_PRG3, LOW);
  digitalWrite(SC_PRG4, LOW);

#ifdef USE_PROG58
  pinMode(SC_PRG5, OUTPUT);
  pinMode(SC_PRG6, OUTPUT);
  pinMode(SC_PRG7, OUTPUT);
  pinMode(SC_PRG8, OUTPUT);
  digitalWrite(SC_PRG5, LOW);
  digitalWrite(SC_PRG6, LOW);
  digitalWrite(SC_PRG7, LOW);
  digitalWrite(SC_PRG8, LOW);
#endif
}

/**
   @brief setup overall program

   - set pins
   - set ROS API
   - reset values
*/
void setup()
{
  set_pins();
  nh.initNode();
  // shared functions
  nh.advertiseService(reset_srv);
  nh.advertise(pub_driver_status);
  nh.subscribe(sub_tool_mode);

  //grinder
  nh.subscribe(grinder_speed_sub);

  //screwdriver
  nh.subscribe(sub_screwdriver_button_cb);
  nh.subscribe(sub_screwdriver_mode_cb);

#ifdef DEBUG
  nh.advertise(pub_debug_data0);
  nh.advertise(pub_debug_data1);
#endif
  reset_values();
}



#ifdef DEBUG
/**
   @brief debugging function
   publish internal states to ROS as needed
*/
void ros_debug() {
  if (screwdriver_on) {
    debug_data0.data = sc_prog;
    if (sc_start) {
      debug_data1.data = 1;
    }
    else if (sc_reverse) {
      debug_data1.data = -1;
    } else if (sc_stop) {
      debug_data1.data = -2;
    }
    pub_debug_data0.publish(&debug_data0);
    pub_debug_data1.publish(&debug_data1);
  } else if (grinder_on) {
    debug_data0.data = RPM;
    debug_data1.data = PWM;
    pub_debug_data0.publish(&debug_data0);
    pub_debug_data1.publish(&debug_data1);
  }
}
#endif
/**
   @brief main loop iteration
   run one cycle of current control Arduino code

   - write data if any set value has been obtained and driver is not in emergency state
   - check against current state
   - reset current state in case of timeout
   - publish driver state
   - publish debug data
   - end cycle by sleep
*/
void loop()
{
  int16_t T_delay = 10;
  int16_t T_MAX = 2000;
  nh.spinOnce();
  driver_state.data = NONE;
  if (t_last_msg >= 0) {
    if (emergency) {
      digitalWrite(VACUUM_SWITCH, LOW);
      digitalWrite(SC_STOP, HIGH);
      analogWrite(GRINDER_SWITCH, 0);
      driver_state.data = OFF;
    } else {
      // write grinder data to arduino
      if (grinder_on) {
        digitalWrite(SC_STOP, HIGH);
        digitalWrite(VACUUM_SWITCH, LOW);
        if (disable_gr()) {
          analogWrite(GRINDER_SWITCH, 0);
          PWM = 0;
        } else {
          analogWrite(GRINDER_SWITCH, PWM);
          driver_state.data = GR;
        }
        // write vacuum data to arduino
      } else if (vacuum_on) {
        digitalWrite(SC_STOP, HIGH);
        analogWrite(GRINDER_SWITCH, 0);
        if (disable_vc()) {
          digitalWrite(VACUUM_SWITCH, LOW);
        } else {
          digitalWrite(VACUUM_SWITCH, HIGH);
          driver_state.data = VC;
        }
      } else if (screwdriver_on) {
        analogWrite(GRINDER_SWITCH, 0);
        digitalWrite(VACUUM_SWITCH, LOW);
        if (disable_sc() or sc_stop) {
          digitalWrite(SC_STOP, HIGH);
        }
        else {
          digitalWrite(SC_STOP, LOW);
          if (sc_prog == 1) {
            digitalWrite(SC_PRG1, HIGH);
          } else {
            digitalWrite(SC_PRG1, LOW);
          }
          if (sc_prog == 2) {
            digitalWrite(SC_PRG2, HIGH);
          } else {
            digitalWrite(SC_PRG2, LOW);
          }
          if (sc_prog == 3) {
            digitalWrite(SC_PRG3, HIGH);
          } else {
            digitalWrite(SC_PRG3, LOW);
          }
          if (sc_prog == 4) {
            digitalWrite(SC_PRG4, HIGH);
          } else {
            digitalWrite(SC_PRG4, LOW);
          }
#ifdef USE_PROG58
          if (sc_prog == 5) {
            digitalWrite(SC_PRG5, HIGH);
          } else {
            digitalWrite(SC_PRG5, LOW);
          }
          if (sc_prog == 6) {
            digitalWrite(SC_PRG6, HIGH);
          } else {
            digitalWrite(SC_PRG6, LOW);
          }
          if (sc_prog == 7) {
            digitalWrite(SC_PRG7, HIGH);
          } else {
            digitalWrite(SC_PRG7, LOW);
          }
          if (sc_prog == 8) {
            digitalWrite(SC_PRG8, HIGH);
          } else {
            digitalWrite(SC_PRG8, LOW);
          }
#endif
          if (sc_start) {
            digitalWrite(SC_START, HIGH);
            digitalWrite(SC_REVERSE, LOW);
          }
          else if (sc_reverse) {
            digitalWrite(SC_START, LOW);
            digitalWrite(SC_REVERSE, HIGH);
          } else {
            digitalWrite(SC_START, LOW);
            digitalWrite(SC_REVERSE, LOW);
          }
          driver_state.data = SC;
        }
      } else {
        digitalWrite(SC_STOP, HIGH);
        digitalWrite(VACUUM_SWITCH, LOW);
        analogWrite(GRINDER_SWITCH, 0);
      }
    }
    t += 1;
  }
  pub_driver_status.publish(&driver_state);
#ifdef DEBUG
  ros_debug();
#endif
  delay(T_delay);
}
